In [14]:
# Copyright 2022 The Forta Foundation
 
import requests
import pandas as pd
import json
import dotenv
import os
from datetime import datetime, timedelta

# Load environment variables
dotenv.load_dotenv()

True

In [2]:
start_date = "2022-12-01" #when the attack detector first started to operate
start_date = "2023-01-10" #when the attack detector first started to operate
attacker_addresses = "0x9d0163e76bbcf776001e639d65f573949a53ab03"
chain = "ethereum" #ethereum or binance 
name = "LendHub"
end_date = "2023-01-14" #should be the date of the attack

In [3]:
alert_hashes_df = pd.DataFrame(columns=["hash","createdAt"])
for attacker_address in attacker_addresses.split(","):
    url = 'https://api.forta.network/graphql'
    query = '''query exampleQuery {
        # first 5 alerts
        alerts(
            input: { first: 200 , addresses: ["'''+attacker_address+'''"],
            blockDateRange: { startDate: "'''+start_date+'''", endDate: "'''+end_date+'''" }
            }
        ) {
            pageInfo {
            hasNextPage
            endCursor {
                alertId
                blockNumber
            }
            }
            alerts {
            hash
            createdAt
            
            }
        }
        }
    '''
    r = requests.post(url, json={'query': query})
    print(r.text)
    json_data = json.loads(r.text)
    df = pd.DataFrame(json_data['data']['alerts']['alerts'])
    print(df)
    alert_hashes_df = pd.concat([alert_hashes_df, df])

len(alert_hashes_df)

{"data":{"alerts":{"pageInfo":{"hasNextPage":false,"endCursor":{"alertId":"","blockNumber":0}},"alerts":[{"hash":"0x5c4bb59bc4757a6ba658be6c13be1f5a4c4e315bfe2965cb72b5cce6b887410c","createdAt":"2023-01-14T09:49:52.195550217Z"},{"hash":"0x414d57b6dadb6a3b8e5ba71d578d1c493ec2bb1bd0d0f52ed2fbc3771ef62025","createdAt":"2023-01-14T09:49:52.150666702Z"},{"hash":"0x5587c4b6974117e3a1c3499fe887929c4161f24c1499b35674d79c0b39a6522f","createdAt":"2023-01-14T09:47:16.813970861Z"},{"hash":"0xd896809a8900d397c80245fb29d1d8ab23b4bf36f2da1112c68c5913580760d2","createdAt":"2023-01-14T09:43:59.114799781Z"},{"hash":"0x72dc20aa34674f046bb65e99c1a2a2696c7dbb80df7769f3ef3e68fdf99ce3d0","createdAt":"2023-01-14T09:41:13.197932279Z"},{"hash":"0x4596e89da5acbe8861ff18518df28d89768b40d147259a9c9af06bda4ae7ac1b","createdAt":"2023-01-14T09:35:57.669903562Z"},{"hash":"0xd5ac9a876dba63f6d3dc5a3ea85595bad9d7f072f762e75946aaf496370456a1","createdAt":"2023-01-13T08:36:43.935917199Z"},{"hash":"0xad03fb24ac3dab11fb65c68

159

In [30]:
start_time = datetime.strptime(alert_hashes_df['createdAt'].min()[0:17]+"00Z","%Y-%m-%dT%H:%M:%SZ")
end_time = datetime.strptime(alert_hashes_df['createdAt'].max()[0:17]+"00Z","%Y-%m-%dT%H:%M:%SZ")


In [33]:
print(datetime.strftime(start_time,"%Y-%m-%dT%H:%M:%SZ"))

2023-01-11T22:11:00Z


In [44]:
current_time = start_time
log_url = "https://api.forta.network/logs/agents/0xac82fb2a572c7c0d41dc19d24790db17148d1e00505596ebe421daf91c837799?minute="
logs = {} # key is scanner node. Logs is a string array
while current_time < end_time:
    request_url = log_url + datetime.strftime(start_time,"%Y-%m-%dT%H:%M:%SZ")
    res = requests.get(request_url)
    lines = res.content.decode("utf-8").split("\n")
    current_scan_node = ""
   
    for line in lines:
        if line[0:2] == "0x":
            current_scan_node = line[0:42]
            if current_scan_node not in logs:
                logs[current_scan_node] = []
        else:
            if current_scan_node == "":
                print(line)
                continue
            logs[current_scan_node].append(line)

    current_time = current_time + timedelta(minutes=1)

KeyboardInterrupt: 

In [40]:
logs

{'0x014fe248922108493e0C724554916c398a9428a0': ['-----------------------------------------------------------------',
  '2023-01-11T22:11:02.187558891Z 2023-01-11 22:11:02,187 - root - DEBUG - handle_alert inner called',
  '2023-01-11T22:11:02.187791403Z 2023-01-11 22:11:02,187 - root - DEBUG - alert 0xdbf7cfdae71bea638e5cbc6f174a10833551bb10ee7899b410eb0a6b5ec30c75 received for incorrect chain None. This bot is for chain 43114.',
  '2023-01-11T22:11:02.187930185Z 2023-01-11 22:11:02,187 - root - DEBUG - Populating luabase cache 2023-01-10 22:11:02.187417',
  '2023-01-11T22:11:02.188057136Z 2023-01-11 22:11:02,187 - root - ERROR - Failed to populate luabase cache 2023-01-10 22:11:02.187417: Invalid chain_id: 43114',
  '2023-01-11T22:11:02.189496871Z 2023-01-11 22:11:02,189 - root - DEBUG - handle_alert called',
  '2023-01-11T22:11:02.189647182Z 2023-01-11 22:11:02,189 - root - DEBUG - handle_alert inner called',
  '2023-01-11T22:11:02.189784703Z 2023-01-11 22:11:02,189 - root - DEBUG - 

In [ ]:
time = start_time

